In [1]:
import pandas as pd
import wikipediaapi
import os
import shutil
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import nltk

In [2]:
def collectBiographiesFromCategories(all_categories, categorymembers, category, data, number_people, level=0, max_level=1): 
    """
    Recursively collects biographical texts and fact graphs for each person in specified Wikipedia category members up to a given recursion level.

    Parameters:
        categorymembers (dict): A dictionary of category member pages from the WikipediaAPI.
        category (str): The name of the category being processed.
        data (list): A list to store biographical text and associated metadata.
        number_people (int): Counter for the number of biographies collected.
        level (int): Current level of recursion through category members.
        max_level (int): Maximum depth of recursion allowed.

    Returns:
        int: Updated count of processed biographies.
    """
    for page in categorymembers.values():
        if page.ns == 0 and 'List' not in page.title and number_people < 130: #limit the number of people for each category to 130 (or more for clustering)

            #some wikipedia page cannot be found in dbpedia, we ignore them
            try: 

                #knowledge graph of facts
                kg_graph = fetchDbpediaFacts(page.pageid, page.title)
                # Save the graph of facts to a JSON file
                #with open('knowledge_graph.json', 'a') as json_file:
                #    json.dump(kg_graph, json_file, indent=4)
                
                all_categories[category].append(kg_graph)
                print(f"{category} {page.title} processed")


                text = page.text
                data.append({'text': text, 'category': category, 'person': (page.title).replace(' ', '')})
                #creation of txt file for the biography
                f = open(f"Biographies_{category}/{(page.title).replace(' ', '')}_{category}.txt", "w", encoding='utf-8')
                f.write(text)
                f.close()

                #keep track of people processed
                number_people += 1  

            except Exception as err:
                print(f"Unexpected {err=}, {type(err)=}, could not process {page.title}")
                continue

            
        if page.ns == wikipediaapi.Namespace.CATEGORY and level < max_level and number_people < 130: #change to 130
            # Recursively call the function to process pages in the subcategory 
            number_people, all_categories = collectBiographiesFromCategories(all_categories, page.categorymembers, category, data, number_people, level=level + 1, max_level=max_level)
    # return updated count of processed people
    return number_people, all_categories

In [3]:
def createBiographyDataFrames():
    """
    Collects biographical data from Wikipedia categories 'Sculptors' and 'Journalists', 
    creates directories for storing these biographies, and formats the collected data into a pandas DataFrame.

    Returns:
        DataFrame: A pandas DataFrame containing the text and category of each biography collected.
    """
    
    
    wiki = wikipediaapi.Wikipedia('Mozilla/5.0', 'en')

    data = []
    all_categories= {}
    all_categories["Journalists"] = []
    all_categories["Sculptors"] = []
    
    if os.path.exists('Biographies'):
        shutil.rmtree('Biographies')
    
    os.mkdir('Biographies')
    os.chdir('Biographies')
        
    sculptors = wiki.page("Category:Sculptors")
    if not os.path.exists('Biographies_Sculptors'):
        os.mkdir('Biographies_Sculptors') #creates a directory to store all sculptors' biographies
    _, all_categories = collectBiographiesFromCategories(all_categories, sculptors.categorymembers, "Sculptors", data, 0)
    
    journalists = wiki.page("Category:Journalists")
    if not os.path.exists('Biographies_Journalists'):
        os.mkdir('Biographies_Journalists') #creates a directory to store all journalists' biographies
    _, all_categories = collectBiographiesFromCategories(all_categories, journalists.categorymembers, "Journalists", data, 0)

    print(all_categories)
    
    with open('knowledge_graph.json', 'w') as json_file:
            json.dump(all_categories, json_file, indent=4)
    
    os.chdir('..')
    
    df = pd.DataFrame(data)
    return df



In [4]:
def fetchDbpediaFacts(wiki_id, page_title):
    """
    Fetches RDF triples (facts) about a specific Wikipedia page from DBpedia using the page's ID.

    Parameters:
        wiki_id (int): The Wikipedia page ID used to fetch the corresponding DBpedia page.
        page_title (str): Title of the Wikipedia page.

    Returns:
        dict: A dictionary containing RDF triples with DBpedia facts about the page, its title and other metadata.
    """

    #initialize the SPARQL wrapper
    sparql = SPARQLWrapper("http://dbpedia.org/sparql/")

    #first query to retrieve the dbpedia page of the person from the wikipedia page id
    sparql.setQuery(f"""
            PREFIX dbo: <http://dbpedia.org/ontology/>
            PREFIX dbr: <http://dbpedia.org/resource/>
            PREFIX dbp: <http://dbpedia.org/property/>

            SELECT *
            WHERE {{
                    ?person dbo:wikiPageID {wiki_id} .
            }}
    """)
    sparql.setReturnFormat(JSON)
    result_link = sparql.query().convert()

    person_wikidata_page = result_link["results"]["bindings"][0]["person"]["value"] #the first value is most likely what we are looking for
    
    #second query to fetch all information as RDF triples
    sparql.setQuery(f"""
            PREFIX dbo: <http://dbpedia.org/ontology/>
            PREFIX dbr: <http://dbpedia.org/resource/>
            PREFIX dbp: <http://dbpedia.org/property/>

            DESCRIBE <{person_wikidata_page}>
    """)
    sparql.setReturnFormat(JSON)
    results_facts = sparql.query().convert()
    
    results_facts['results']['bindings'] = [entry for entry in results_facts['results']['bindings']
        if ('lang' not in entry['o'].keys() or entry['o']['lang'] == 'en')
        and ('lang' not in entry['p'].keys() or entry['p']['lang'] == 'en')
        and ('lang' not in entry['s'].keys() or entry['s']['lang'] == 'en')][:100]  #first 100 english facts only

    results_facts['head']['person'] = page_title.replace(" ", "")
    
    return(results_facts)

print(os.getcwd())